# 安然数据集分析工作文档

### 问题1
#### 向我们总结此项目的目标以及机器学习对于实现此目标有何帮助。作为答案的部分，提供一些数据集背景信息以及这些信息如何用于回答项目问题。你在获得数据时它们是否包含任何异常值，你是如何进行处理的？【相关标准项：“数据探索”，“异常值调查”】 

数据集中含有大量安然公司高管的邮件和财务数据信息，本项目旨在借助机器学习算法对这些数据进行分析，探索涉案人员的特征，从而找出有欺诈嫌疑的安然员工。     

调用函数describe_dataset()可查看数据集基本情况，数据集共包含146条记录（人），包括1个标签poi和20个特征，有18个人被标记为'poi'（person of interest），128人不是'poi'。   

随机调取数据集中记录可以看到有些特征的值缺失（NaN），通过函数count_nans()统计各特征缺失情况。特征'total_payments'和'total_stock_value'的缺失值数量最少，分别作为横纵坐标绘制散点图，发现两个点的'total_payments'值特别高，查看具体记录发现分别为'LAY KENNETH L'和'TOTAL','LAY KENNETH L'是公司最高领导人，也属于'poi'，记录保留，而'TOTAL'明显不是人名，删除该记录。 

### 问题2
#### 你最终在你的 POI 标识符中使用了什么特征，你使用了什么筛选过程来挑选它们？你是否需要进行任何缩放？为什么？作为任务的一部分，你应该尝试设计自己的特征，而非使用数据集中现成的——解释你尝试创建的特征及其基本原理。（你不一定要在最后的分析中使用它，而只设计并测试它）。在你的特征选择步骤，如果你使用了算法（如决策树），请也给出所使用特征的特征重要性；如果你使用了自动特征选择函数（如 SelectBest），请报告特征得分及你所选的参数值的原因。【相关标准项：“创建新特征”、“适当缩放特征”、“智能选择功能”】  

创建了新的特征'fraction_from_poi'和'fraction_to_poi'，分别表示与poi人员往来邮件数占总往来邮件数的比例，由'to_messages', 
'from_poi_to_this_person'和'from_messages', 'from_this_person_to_poi'之间的计算得出。

最终使用朴素贝叶斯算法。先使用所有缺失值比例不高于66.7%的特征参与朴素贝叶斯算法模型构建，性能评估结果为：
```
Accuracy: 0.83093	Precision: 0.30939	Recall: 0.21750	
```
通过SelectKBest选择出5个特征，包括： 'salary','exercised_stock_options','bonus','total_stock_value','fraction_to_poi'，特征得分分别为：
```
salary 16.9609162406
exercised_stock_options 22.846900557
bonus 15.4914145509
total_stock_value 22.334566139
fraction_to_poi 13.8058968665
```
选用这5个特征后算法性能明显提高。

使用决策树算法时，最初使用的也是以上5个特征，得出的特征重要性为：
```
[ 0.13514851  0.2449697   0.29368354  0.06223037  0.26396787]
```
删去重要性较低的'total_stock_value'后，决策树算法评估结果有所改善，决策树算法最终使用4个特征：'salary', 'exercised_stock_options', 'bonus', 'fraction_to_poi'。

### 问题3
#### 你最终使用了什么算法？你还尝试了其他什么算法？不同算法之间的模型性能有何差异？【相关标准项：“选择算法”】

尝试了朴素贝叶斯算法、决策树算法、支持向量机算法，优化参数后朴素贝叶斯算法模型性能最好，支持向量机性能最差。   
朴素贝叶斯性能评估结果：
```
Accuracy: 0.85629	Precision: 0.49545	Recall: 0.32650	
```
决策树性能评估结果：
```
Accuracy: 0.79446	Precision: 0.34328	Recall: 0.36800	
```
支持向量机性能评估结果：
```
Accuracy: 0.85736	Precision: 0.50857	Recall: 0.04450
```


### 问题4
#### 调整算法的参数是什么意思，如果你不这样做会发生什么？你是如何调整特定算法的参数的？（一些算法没有需要调整的参数 – 如果你选择的算法是这种情况，指明并简要解释对于你最终未选择的模型或需要参数调整的不同模型，例如决策树分类器，你会怎么做）。【相关标准项：“调整算法”】
不同的算法原理不同，涉及许多不同的参数，调整参数能够影响算法的表现，需要把握好偏差和方差的平衡，在避免过拟合的前提下尽量提高算法的预测能力。   

使用决策树算法时尝试手动调整了min_samples_split参数，发现min_samples_split=0时性能相对较高，使用支持向量机算法时借助了GridSearchCV自动搜索最佳参数搭配。

### 问题5
#### 什么是验证，未正确执行情况下的典型错误是什么？你是如何验证你的分析的？【相关标准项：“验证策略”】
验证是对于所构建的方法的性能的评估，通常使用测试集来验证性能。典型错误是使用全部数据参与模型训练，这样会导致过拟合，训练出的模型泛化能力差。
我借助了tester.py中的test_classifier()方法进行验证。由于安然数据集样本量很小，使用sklearn.model_selection模块中的 StratifiedShuffleSplit，进行了1000次迭代取样，分别计算accuracy、precision、recall等。

### 问题6
####  给出至少 2 个评估度量并说明每个的平均性能。解释对用简单的语言表明算法性能的度量的解读。【相关标准项：“评估度量的使用”】
本次分析主要依据precision和recall进行评分，最终选用的朴素贝叶斯算法评估结果：
```
GaussianNB(priors=None)
	Accuracy: 0.85629	Precision: 0.49545	Recall: 0.32650	F1: 0.39361	F2: 0.35040
	Total predictions: 14000	True positives:  653	False positives:  665	False negatives: 1347	True negatives: 11335
```
precision针对的是预测结果，表示的是预测为poi的样本中有多少预测是正确的,即true_positives/(true_positives+false_positives)。   
recall是针对样本而言的，表示所有样本中被正确预测的poi样本数，即
true_positives/(true_positives+false_negatives)。

### 参考信息
我在此确认，所提交的项目为我的工作成果，以下为我在完成项目过程中所参考的网站、书籍、论坛、博客文章和 GitHub 代码库。    
* [scikit-learn文档](http://scikit-learn.org/stable/index.html)
* [accuracy、precision、recall的理解](https://link.zhihu.com/?target=http%3A//charleshm.github.io/2016/03/Model-Performance/)
* [机选与手选特征，其评价结果不一致的解释](https://discussions.youdaxue.com/t/p5/46193/2)
* [GridSearchCV、pipeline使用](https://discussions.youdaxue.com/t/gridsearchcv/46395/2)
